In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [142]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [143]:
obs = pd.read_csv("../data/obs_windows/observation_72H_prediction_24H_48H_72H.csv")

C:\Users\Conrad\AppData\Local\Temp\ipykernel_9100\3718084596.py:1: DtypeWarning: Columns (3,4,7,8,10,11,12,13,14,15,16,17,19,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  obs = pd.read_csv("../data/obs_windows/observation_72H_prediction_24H_48H_72H.csv")


### 7d observation window with 24hr prediction window

In [144]:
obs_24 = obs.drop(obs.loc[:, 'spn':'full_derate_within_24hr'], axis=1).drop(obs.loc[:, 'partial_derate_within_48hr':'derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [145]:
eq_ids = obs_24.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [146]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [147]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [148]:
#merge modeling column onto original dataframe
obs_24 = obs_24.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_24 = obs_24.merge(eq_ids, on='EquipmentID')
obs_24 = obs_24.drop(columns='EquipmentID')

In [149]:
#creating training, testing, and validation dfs
obs_24_training = obs_24[obs_24['modeling']=='training']
obs_24_training = obs_24_training.drop(columns=['modeling', 'random'])
print(obs_24_training.shape)

obs_24_test = obs_24[obs_24['modeling']=='testing']
obs_24_test = obs_24_test.drop(columns=['modeling', 'random'])
print(obs_24_test.shape)

obs_24_validation = obs_24[obs_24['modeling']=='validation']
obs_24_validation = obs_24_validation.drop(columns=['modeling', 'random'])
print(obs_24_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [150]:
df_sampling = obs_24_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_24hr'] == 0][0:18855]

In [151]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_24hr']==1], df_sampling_non_tenth])

In [152]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_24hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

In [153]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [154]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_24hr']==1]
smote_derate.shape

(18855, 56)

In [155]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_24hr']==0]])
und_samp.shape

(215061, 56)

In [156]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_24hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

X_test = obs_24_test[features]
y_test = obs_24_test[target]

In [157]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [158]:
print(y_resampled.shape)

(56565, 1)


In [159]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [160]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [161]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,3.349359e-01
0,Aftertreatment System,2.631610e-01
51,Unknown,1.211310e-01
18,Engine Coolant,7.195539e-02
22,Engine Fuel,3.319269e-02
28,Engine Protection Torque Full Derate,2.577222e-02
43,J1939 Network,2.237719e-02
41,Intake Manifold,2.209986e-02
52,Variable Geometry Turbocharger,1.978333e-02
39,Injector Metering Rail,1.519826e-02


In [162]:
confusion_matrix(y_test, y_pred)

array([[190193,   7819],
       [  1192,   3545]], dtype=int64)

In [163]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [164]:
(tn, fp, fn, tp)

(190193, 7819, 1192, 3545)

In [165]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98    198012
         1.0       0.31      0.75      0.44      4737

    accuracy                           0.96    202749
   macro avg       0.65      0.85      0.71    202749
weighted avg       0.98      0.96      0.96    202749



In [166]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [167]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9189162931001326


### 7d observation window with 48hr prediction window

In [168]:
obs_48 = obs.drop(obs.loc[:, 'spn':'full_derate_within_48hr'], axis=1).drop(obs.loc[:, 'partial_derate_within_72hr':'derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [169]:
eq_ids = obs_48.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [170]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [171]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [172]:
#merge modeling column onto original dataframe
obs_48 = obs_48.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_48 = obs_48.merge(eq_ids, on='EquipmentID')
obs_48 = obs_48.drop(columns='EquipmentID')

In [173]:
#creating training, testing, and validation dfs
obs_48_training = obs_48[obs_48['modeling']=='training']
obs_48_training = obs_48_training.drop(columns=['modeling', 'random'])
print(obs_48_training.shape)

obs_48_test = obs_48[obs_48['modeling']=='testing']
obs_48_test = obs_48_test.drop(columns=['modeling', 'random'])
print(obs_48_test.shape)

obs_48_validation = obs_48[obs_48['modeling']=='validation']
obs_48_validation = obs_48_validation.drop(columns=['modeling', 'random'])
print(obs_48_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [174]:
df_sampling = obs_48_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_48hr'] == 0][0:18855]

In [175]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_48hr']==1], df_sampling_non_tenth])

In [176]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_48hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

X_test = obs_48_test[features]
y_test = obs_48_test[target]

In [177]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [178]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_48hr']==1]
smote_derate.shape

(18855, 56)

In [179]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_48hr']==0]])
und_samp.shape

(214184, 56)

In [180]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_48hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

In [181]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [182]:
print(y_resampled.shape)

(56565, 1)


In [183]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [184]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [185]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,3.162794e-01
0,Aftertreatment System,2.526229e-01
51,Unknown,1.370247e-01
18,Engine Coolant,6.998638e-02
22,Engine Fuel,3.963649e-02
43,J1939 Network,2.811492e-02
28,Engine Protection Torque Full Derate,2.301806e-02
52,Variable Geometry Turbocharger,1.879153e-02
41,Intake Manifold,1.855502e-02
39,Injector Metering Rail,1.666071e-02


In [186]:
confusion_matrix(y_test, y_pred)

array([[189720,   7608],
       [  1683,   3738]], dtype=int64)

In [187]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [188]:
(tn, fp, fn, tp)

(189720, 7608, 1683, 3738)

In [189]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98    197328
         1.0       0.33      0.69      0.45      5421

    accuracy                           0.95    202749
   macro avg       0.66      0.83      0.71    202749
weighted avg       0.97      0.95      0.96    202749



In [190]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [191]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8927384779488124


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 7d observation window with 72hr prediction window

In [192]:
obs_72 = obs.drop(obs.loc[:, 'spn':'full_derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [193]:
eq_ids = obs_72.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [194]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [195]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [196]:
#merge modeling column onto original dataframe
obs_72 = obs_72.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_72 = obs_72.merge(eq_ids, on='EquipmentID')
obs_72 = obs_72.drop(columns='EquipmentID')

In [197]:
#creating training, testing, and validation dfs
obs_72_training = obs_72[obs_72['modeling']=='training']
obs_72_training = obs_72_training.drop(columns=['modeling', 'random'])
print(obs_72_training.shape)

obs_72_test = obs_72[obs_72['modeling']=='testing']
obs_72_test = obs_72_test.drop(columns=['modeling', 'random'])
print(obs_72_test.shape)

obs_72_validation = obs_72[obs_72['modeling']=='validation']
obs_72_validation = obs_72_validation.drop(columns=['modeling', 'random'])
print(obs_72_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [198]:
df_sampling = obs_72_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_72hr'] == 0][0:18855]

In [199]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_72hr']==1], df_sampling_non_tenth])

In [200]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_72hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

In [201]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [202]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_72hr']==1]
smote_derate.shape

(18855, 56)

In [203]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_72hr']==0]])
und_samp.shape

(213477, 56)

In [204]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_72hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

X_test = obs_72_test[features]
y_test = obs_72_test[target]

In [205]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [206]:
print(y_resampled.shape)

(56565, 1)


In [207]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [208]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [209]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,0.312112
0,Aftertreatment System,0.249037
51,Unknown,0.140826
18,Engine Coolant,0.067073
22,Engine Fuel,0.041008
43,J1939 Network,0.030232
28,Engine Protection Torque Full Derate,0.023944
41,Intake Manifold,0.019807
52,Variable Geometry Turbocharger,0.016838
17,Engine Control Module,0.016589


In [210]:
confusion_matrix(y_test, y_pred)

array([[188107,   8744],
       [  1940,   3958]], dtype=int64)

In [211]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [212]:
(tn, fp, fn, tp)

(188107, 8744, 1940, 3958)

In [213]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97    196851
         1.0       0.31      0.67      0.43      5898

    accuracy                           0.95    202749
   macro avg       0.65      0.81      0.70    202749
weighted avg       0.97      0.95      0.96    202749



In [214]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [215]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8750448238853401


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
